RESTORE

[algospot의 RESTORE 링크](https://algospot.com/judge/problem/read/RESTORE)
  
### 문제  
토요일에 출근해서 연구실에서 놀고 있던 대학원생 진호는 실수로 실험에 사용하던 데이터를 삭제하고 말았습니다. 복사본도 없는 터라 이대로라면 교수님의 진노를 한 몸에 받을 것은 자명한 일, 따라서 진호는 그럴 듯해 보이는 데이터를 위조하여 교수님의 분노를 피해 가기로 합니다. 우리가 데이터에 대해 알고있는 것은, 데이터가 k개의 문자열 조각을 부분 문자열로 포함하며, 모두 알파벳 소문자로 구성된다는 사실 밖에 없습니다. (어떤 문자열의 부분 문자열은 해당 문자열의 연속된 일부분입니다.)  
  
주어진 문자열 조각들을 모두 부분 문자열로 포함하는 문자열 중 가장 짧은 것을 계산하는 프로그램을 작성하세요. 만약 이와 같은 문자열이 여럿이라면 아무 문자열이나 출력하면 됩니다.  
   
### 출력   
한 줄로, 해당 문자열을 모두 포함하는 가장 짧은 문자열 중 하나를 출력합니다.      

solution의  
pieces : 문자열 조각의 리스트

In [1]:
from copy import deepcopy

def solution(pieces, mem = None, overlap=None):
    if mem is None : # 메모이재이션 초기화
        remove_list = []
        for i in range(len(pieces)):
            for j in range(len(pieces)):
                if i != j and pieces[j] in pieces[i]:
                    remove_list.append(pieces[j])
        for i in remove_list: 
            pieces.remove(i) # 포함되는 단어들은 제거

        mem = [[-1 for _ in range(1 << len(pieces))] for _ in range(len(pieces))] # 메모이재이션 리스트
        overlap = [[0 for _ in range(len(pieces))] for _ in range(len(pieces))] # overlap[단어1][단어2] = 단어1과 2 사이에 겹치는 글자수
        for i in range(len(pieces)):
            for j in range(len(pieces)):
                if i == j:
                    continue
                else:
                    index = len(pieces[i])-1
                    for k in range(len(pieces[j])-1, -1, -1):
                        if pieces[j][k] == pieces[i][index]:
                            overlap[i][j] += 1
                            index -= 1
                        else:
                            overlap[i][j] = 0
        # overlap되는 단어 수를 저장해줌.
    answer = pieces[0] + reconstruct(len(pieces), pieces, 0, 1 << 0, mem, overlap)
    for i in range(1, len(pieces)):
        temp_answer = pieces[1] + reconstruct(len(pieces), pieces, i, 1 << i, mem, overlap) 
        if len(answer) > len(temp_answer):
            answer = temp_answer # 첫번째부터 끝까지 최소 길이 단어를 찾음
    return answer # 최소 길이 단어 반환
        
def restore(pieces_num, last, used, mem, overlap):
    if used == (1 << pieces_num)-1 : return 0 # 만약 모든 부분단어를 모두 사용했을 경우 ex) 부분단어 3개 -> 111 일경우 모두 사용한것임
    if mem[last][used] != -1: 
        return mem[last][used] # 메모이재이션으로 이미 저장한 것은 바로 리턴
    mem[last][used] = 0 # 메모이재이션 초기화
    for next_one in range(pieces_num):
        if used & (1<< next_one) == 0: # 만약 next_one 번째 단어가 아직 쓰이지 않았다면
            mem[last][used] = max(mem[last][used], overlap[last][next_one] + restore(pieces_num ,next_one, used + (1 << next_one), mem, overlap)
            ) # 쓰이지 않은 단어들 중에 가장 많은 overlap을 반환해주는 것을 사용
    return mem[last][used] # 그리고 반환해줌

def reconstruct(pieces_num, pieces, last, used, mem, overlap):
    if used == (1 << pieces_num) -1: # 만약 모든 단어가 사용된 상태라면 리턴 (base case)
        return ""
    for next_one in range(pieces_num): # 모든 부분 단어에 대해서
        if used & (1 << next_one): continue # 쓰인 부분단어라면 다음 단어로 넘김
        ifUsed = restore(pieces_num, next_one, used + (1<<next_one), mem, overlap) + overlap[last][next_one]
        if restore(pieces_num, last, used, mem, overlap) == ifUsed: # 만약 최적해와 같은 답이 나온다면 나중에 나온 답으로 바꿔줌
            return pieces[next_one][overlap[last][next_one]:] + reconstruct(pieces_num, pieces, next_one, used+(1<<next_one), mem, overlap)
    return "Error!!!" # 이쪽으로 나오면 에러

In [2]:
pieces = ['geo','oji','jing']

In [3]:
solution(pieces)

'geojing'

In [4]:
pieces = ['world','hello']

In [5]:
solution(pieces)

'worldhello'

In [6]:
pieces = ['abrac', 'cadabra', 'dabr']

In [7]:
solution(pieces)

'cadabrac'